In [1]:
from os import path,listdir
from shutil import move
from datetime import datetime,timedelta
import time
import re
import pandas as pd
import numpy as np
import random
import pytz
import json
from tqdm import tqdm
local_tz = 'Asia/Almaty'

In [2]:
fn_matches='data/matches.csv'
fn_done='data/matches_done.csv'
df_matches=pd.read_csv(fn_matches, index_col=None)
df_done=pd.read_csv(fn_done, index_col=None)
#SERVER_TZ = 'UTC'
#df_matches['ts']=pd.DatetimeIndex(pd.to_datetime(df_matches['startTimestamp'], unit='s')).tz_localize(SERVER_TZ)
df_matches=df_matches.drop_duplicates(keep='first')
df=pd.concat([df_done,df_matches.loc[df_matches['done']==1]])
df_matches=df_matches.loc[df_matches['done']==0]
df.to_csv('data/matches2.csv', index=False)
df_matches.to_csv('data/matches1.csv', index=False)

In [13]:
df=pd.read_csv(fn_matches, index_col=None)
df['ts']=pd.to_datetime(df['ts'])
ds=pytz.timezone(local_tz).localize(datetime(2020, 7, 10))
de=pytz.timezone(local_tz).localize(datetime(2020, 11, 1))
df=df[(df['ts']>ds) & (df['ts']<de)]
df=df.drop_duplicates()
df.to_csv('data/matches.csv', index=False)

In [14]:
df

,homeTeam,status,homeTeamShort,homeScoreHT,homeScoreFT,awayTeamShort,tournament,awayTeam,ts,awayScoreFT,awayScoreHT,id,awayScoreET,round,winnerCode,coverage,done,startTimestamp,country,homeScoreET
1503,Everton,100,Everton,0.0,1.0,Aston Villa,premier-league,Aston Villa,2020-07-16 17:00:00+00:00,1.0,0.0,8692266,1.0,36.0,3,1,0,1594918800,england,1.0
1504,Leicester City,100,Leicester,1.0,2.0,Sheffield Utd,premier-league,Sheffield United,2020-07-16 17:00:00+00:00,0.0,0.0,8692268,0.0,36.0,1,1,0,1594918800,england,2.0
1505,Crystal Palace,100,Crystal Palace,0.0,0.0,Man Utd,premier-league,Manchester United,2020-07-16 19:15:00+00:00,2.0,1.0,8692264,2.0,36.0,2,1,0,1594926900,england,0.0
1506,Southampton,100,Southampton,0.0,1.0,Brighton,premier-league,Brighton & Hove Albion,2020-07-16 19:15:00+00:00,1.0,1.0,8692262,1.0,36.0,3,1,0,1594926900,england,1.0
1507,West Ham United,100,West Ham,3.0,3.0,Watford,premier-league,Watford,2020-07-17 19:00:00+00:00,1.0,0.0,8692271,1.0,36.0,1,1,0,1595012400,england,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156548,FC Vizela,100,Vizela,0.0,1.0,Chaves,segunda-liga,GD Chaves,2020-10-05 18:45:00+00:00,1.0,0.0,8956665,1.0,5.0,3,3,0,1601923500,portugal,1.0
156549,UTA Arad,100,UTA Arad,0.0,0.0,Academica C.,liga-i,FC Academica Clinceni,2020-10-05 18:00:00+00:00,0.0,0.0,8849122,0.0,6.0,3,1,0,1601920800,romania,0.0
156550,Degerfors IF,100,Degerfors,3.0,4.0,Västerås,superettan,Västerås SK,2020-10-05 17:00:00+00:00,1.0,1.0,8731037,1.0,21.0,1,3,0,1601917200,sweden,4.0
156551,Trelleborgs FF,100,Trelleborg,1.0,2.0,Norrby,superettan,Norrby IF,2020-10-05 17:00:00+00:00,2.0,2.0,8731023,2.0,21.0,3,3,0,1601917200,sweden,2.0


In [3]:
def parse_fn(fn):
    return int(fn.split('_')[1][:-5])

def parse_votes(data, mid):
    row=data['vote']
    row['mid']=mid
    return row

def parse_graph(data, mid):
    rows=[{'mid':mid,'minute':x['minute'],'value':x['value'] } for x in data['graphPoints']]
    return rows

def parse_incidents(data, mid):
    rows=[]
    for x in data[::-1]:
        row={'mid':mid}
        if 'player' in x.keys():
            row['player1']=x['player']['name']
            if 'position' in x['player'].keys():
                row['position1']=x['player']['position']
        if 'assist1' in x.keys():
            row['player2']=x['assist1']['name']
            if 'position' in x['assist1'].keys():
                row['position2']=x['assist1']['position']
        if 'playerIn' in x.keys():
            row['player1']=x['playerIn']['name']
            if 'position' in x['playerIn'].keys():
                row['position1']=x['playerIn']['position']
        if 'playerOut' in x.keys():
            row['player2']=x['playerOut']['name']
            if 'position' in x['playerOut'].keys():
                row['position2']=x['playerOut']['position']
        if  'incidentClass' in x.keys() and x['incidentType']=='card':
            row['incidentType']=x['incidentClass']
        else:
            row['incidentType']=x['incidentType']
        if 'addedTime' in x.keys():
            row['addedTime']=x['length'] if x['incidentType']=='injuryTime' else x['addedTime']
        if 'isHome' in x.keys():
            row['isHome']=x['isHome'] 
        if 'time' in x.keys():
            row['time']=x['time']
        rows.append(row)
    return rows

def parse_team(data, home, mid):
    rows= [  {
        'mid':mid,
        'home':home,
        'name' : x['player']['name'],
        'slug' : x['player']['slug'],
        'position' : x['position'] if 'position' in x.keys() else '',
        'substitute' : x['substitute'] if 'substitute' in x.keys() else False,
        'totalPass' : x['statistics']['totalPass'] if 'statistics' in x.keys() and 'totalPass' in x['statistics'].keys() else np.NaN,
        'accuratePass' : x['statistics']['accuratePass'] if 'statistics' in x.keys() and 'accuratePass' in x['statistics'].keys() else np.NaN,
        'totalLongBalls' : x['statistics']['totalLongBalls'] if 'statistics' in x.keys() and 'totalLongBalls' in x['statistics'].keys() else np.NaN,
        'accurateLongBalls' : x['statistics']['accurateLongBalls'] if 'statistics' in x.keys() and 'accurateLongBalls' in x['statistics'].keys() else np.NaN,
        'aerialLost' : x['statistics']['aerialLost'] if 'statistics' in x.keys() and 'aerialLost' in x['statistics'].keys() else np.NaN,
        'duelLost' : x['statistics']['duelLost'] if 'statistics' in x.keys() and 'duelLost' in x['statistics'].keys() else np.NaN,
        'totalClearance' : x['statistics']['totalClearance'] if 'statistics' in x.keys() and 'totalClearance' in x['statistics'].keys() else np.NaN,
        'goodHighClaim' : x['statistics']['goodHighClaim'] if 'statistics' in x.keys() and 'goodHighClaim' in x['statistics'].keys() else np.NaN,
        'savedShotsFromInsideTheBox' : x['statistics']['savedShotsFromInsideTheBox'] if 'statistics' in x.keys() and 'tosavedShotsFromInsideTheBoxalCross' in x['statistics'].keys() else np.NaN,
        'saves' : x['statistics']['saves'] if 'statistics' in x.keys() and 'saves' in x['statistics'].keys() else np.NaN,
        'totalKeeperSweeper' : x['statistics']['totalKeeperSweeper'] if 'statistics' in x.keys() and 'totalKeeperSweeper' in x['statistics'].keys() else np.NaN,
        'accurateKeeperSweeper' : x['statistics']['accurateKeeperSweeper'] if 'statistics' in x.keys() and 'accurateKeeperSweeper' in x['statistics'].keys() else np.NaN,
        'minutesPlayed' : x['statistics']['minutesPlayed'] if 'statistics' in x.keys() and 'minutesPlayed' in x['statistics'].keys() else np.NaN,
        'touches' : x['statistics']['touches'] if 'statistics' in x.keys() and 'touches' in x['statistics'].keys() else np.NaN,
        'rating' : x['statistics']['rating'] if 'statistics' in x.keys() and 'rating' in x['statistics'].keys() else np.NaN,
        'totalCross' : x['statistics']['totalCross'] if 'statistics' in x.keys() and 'totalCross' in x['statistics'].keys() else np.NaN,
        'aerialWon' : x['statistics']['aerialWon'] if 'statistics' in x.keys() and 'aerialWon' in x['statistics'].keys() else np.NaN,
        'duelWon' : x['statistics']['duelWon'] if 'statistics' in x.keys() and 'duelWon' in x['statistics'].keys() else np.NaN,
        'interceptionWon' : x['statistics']['interceptionWon'] if 'statistics' in x.keys() and 'interceptionWon' in x['statistics'].keys() else np.NaN,
        'totalTackle' : x['statistics']['totalTackle'] if 'statistics' in x.keys() and 'totalTackle' in x['statistics'].keys() else np.NaN,
        'fouls' : x['statistics']['fouls'] if 'statistics' in x.keys() and 'fouls' in x['statistics'].keys() else np.NaN,
        'shotOffTarget' : x['statistics']['shotOffTarget'] if 'statistics' in x.keys() and 'shotOffTarget' in x['statistics'].keys() else np.NaN,
        'wasFouled' : x['statistics']['wasFouled'] if 'statistics' in x.keys() and 'wasFouled' in x['statistics'].keys() else np.NaN,
        'onTargetScoringAttempt' : x['statistics']['onTargetScoringAttempt'] if 'statistics' in x.keys() and 'onTargetScoringAttempt' in x['statistics'].keys() else np.NaN,
        'blockedScoringAttempt' : x['statistics']['blockedScoringAttempt'] if 'statistics' in x.keys() and 'blockedScoringAttempt' in x['statistics'].keys() else np.NaN,
        'wonContest' : x['statistics']['wonContest'] if 'statistics' in x.keys() and 'wonContest' in x['statistics'].keys() else np.NaN,
        'totalContest' : x['statistics']['totalContest'] if 'statistics' in x.keys() and 'totalContest' in x['statistics'].keys() else np.NaN,
        'challengeLost' : x['statistics']['challengeLost'] if 'statistics' in x.keys() and 'challengeLost' in x['statistics'].keys() else np.NaN,
        'dispossessed' : x['statistics']['dispossessed'] if 'statistics' in x.keys() and 'dispossessed' in x['statistics'].keys() else np.NaN,
        'savedShotsFromInsideTheBox' : x['statistics']['savedShotsFromInsideTheBox'] if 'statistics' in x.keys() and 'savedShotsFromInsideTheBox' in x['statistics'].keys() else np.NaN,
        'totalOffside' : x['statistics']['totalOffside'] if 'statistics' in x.keys() and 'totalOffside' in x['statistics'].keys() else np.NaN,
        'bigChanceCreated' : x['statistics']['bigChanceCreated'] if 'statistics' in x.keys() and 'bigChanceCreated' in x['statistics'].keys() else np.NaN,
        'goals' : x['statistics']['goals'] if 'statistics' in x.keys() and 'goals' in x['statistics'].keys() else np.NaN }  
    for x in data['players'] ]
    return rows

def parse_lineups(data, mid):
    home=data['home']
    away=data['away']
    formations= { 
        'mid':mid, 
        'formation_h': home['formation'] if 'formation' in home.keys() else '', 
        'formation_a': away['formation'] if 'formation' in away.keys() else '' }
    rows=parse_team(home, 1, mid)
    rows+=parse_team(away, 0, mid)
    return rows, formations

def parse_htft(x, mid):
    home_row= { 'mid':mid, 'period':x['period'], 'ishome':1 }
    for i in x['groups']:
        for j in i['statisticsItems']:
            col=j['name']
            home_row[col]= j['home']
    away_row= { 'mid':mid, 'period':x['period'], 'ishome':0 }
    for i in x['groups']:
        for j in i['statisticsItems']:
            col=j['name']
            away_row[col]= j['away']
    return home_row,away_row

def parse_matches():
    DIR='raw/matches/'
    DIR_OUT='raw/done/matches/'
    arr = {'votes':[],
        'graph':[],
        'incidents':[],
        'lineups':[],
        'statistics':[],
        'formations':[]  }
    row={}
    for file in tqdm(listdir('raw/matches')):
        if len(file)>25 or file=='.empty':
            continue
        mid=int(file.split('_')[1][:-5])
        with open(DIR+file, 'r', encoding='utf8') as f:
            data=json.load(f)
        isFound=True
        if file.startswith('votes'):
            row=parse_votes(data,mid)
            arr['votes'].append(row)
        elif file.startswith('graph'):
            rows=parse_graph(data,mid)
            arr['graph']+=rows
        elif file.startswith('incidents'):
            rows=parse_incidents(data['incidents'],mid)
            arr['incidents']+=rows
        elif file.startswith('lineups'):
            rows, formations=parse_lineups(data,mid)
            if formations['formation_h']!='' and formations['formation_a']!='':
                arr['formations'].append(formations) 
            arr['lineups']+=rows
        elif file.startswith('statistics'):
            for x in data['statistics']:
                home_row,away_row=parse_htft(x,mid)
                arr['statistics'].append(home_row)
                arr['statistics'].append(away_row)
        else:
            isFound=False
        if isFound:
            move( DIR+file, DIR_OUT+file)
    
    name='votes'
    file_name=f'data/{name}.csv'
    if path.exists(file_name):
        pd.concat([pd.read_csv(file_name, index_col=None), pd.DataFrame(data=arr[name])]).to_csv(file_name, index=False)
    else:
        pd.DataFrame(data=arr[name]).to_csv(file_name, index=False)
    
    name='graph'
    file_name=f'data/{name}.csv'
    if path.exists(file_name):
        pd.concat([pd.read_csv(file_name, index_col=None), pd.DataFrame(data=arr[name])]).to_csv(file_name, index=False)
    else:
        pd.DataFrame(data=arr[name]).to_csv(file_name, index=False)
    
    name='incidents'
    file_name=f'data/{name}.csv'
    if path.exists(file_name):
        pd.concat([pd.read_csv(file_name, index_col=None), pd.DataFrame(data=arr[name])]).to_csv(file_name, index=False)
    else:
        pd.DataFrame(data=arr[name]).to_csv(file_name, index=False)
    
    name='lineups'
    file_name=f'data/{name}.csv'
    if path.exists(file_name):
        pd.concat([pd.read_csv(file_name, index_col=None), pd.DataFrame(data=arr[name])]).to_csv(file_name, index=False)
    else:
        pd.DataFrame(data=arr[name]).to_csv(file_name, index=False)
    
    name='formations'
    file_name=f'data/{name}.csv'
    if path.exists(file_name):
        pd.concat([pd.read_csv(file_name, index_col=None), pd.DataFrame(data=arr[name])]).to_csv(file_name, index=False)
    else:
        pd.DataFrame(data=arr[name]).to_csv(file_name, index=False)
    
    name='statistics'
    file_name=f'data/{name}.csv'
    if path.exists(file_name):
        pd.concat([pd.read_csv(file_name, index_col=None), pd.DataFrame(data=arr[name])]).to_csv(file_name, index=False)
    else:
        pd.DataFrame(data=arr[name]).to_csv(file_name, index=False)
    


In [4]:
parse_matches()

100%|██████████| 22127/22127 [02:10<00:00, 169.57it/s]


In [ ]:
row

In [ ]:
file_name='raw/matches/graph_5076016.json'
with open(file_name, 'r', encoding='utf8') as f:
    data=json.load(f)
a=data["graphPoints"]

In [ ]:
pd.DataFrame(data=[row])

In [ ]:
 row={}
 for x in a:
     row['m{}'.format(x['minute'])]=x['value']
row 

In [ ]:

file_name='raw/matches/statistics_6768144.json'
with open(file_name, 'r', encoding='utf8') as f:
    data=json.load(f)

In [ ]:
rows=[]
for x in data['statistics']:
    home_row,away_row=parse_htft(x,1)
    rows.append(home_row)
    rows.append(away_row)
rows

In [ ]:

[  {
    'name' : x['player']['name'],
    'slug' : x['player']['slug'],
    'position' : x['position'] if 'position' in x.keys() else '',
    'substitute' : x['substitute'] if 'substitute' in x.keys() else False,
    'totalPass' : x['statistics']['totalPass'] if 'statistics' in x.keys() and 'totalPass' in x['statistics'].keys() else np.NaN,
    'accuratePass' : x['statistics']['accuratePass'] if 'statistics' in x.keys() and 'accuratePass' in x['statistics'].keys() else np.NaN,
    'totalLongBalls' : x['statistics']['totalLongBalls'] if 'statistics' in x.keys() and 'totalLongBalls' in x['statistics'].keys() else np.NaN,
    'accurateLongBalls' : x['statistics']['accurateLongBalls'] if 'statistics' in x.keys() and 'accurateLongBalls' in x['statistics'].keys() else np.NaN,
    'aerialLost' : x['statistics']['aerialLost'] if 'statistics' in x.keys() and 'aerialLost' in x['statistics'].keys() else np.NaN,
    'duelLost' : x['statistics']['duelLost'] if 'statistics' in x.keys() and 'duelLost' in x['statistics'].keys() else np.NaN,
    'totalClearance' : x['statistics']['totalClearance'] if 'statistics' in x.keys() and 'totalClearance' in x['statistics'].keys() else np.NaN,
    'goodHighClaim' : x['statistics']['goodHighClaim'] if 'statistics' in x.keys() and 'goodHighClaim' in x['statistics'].keys() else np.NaN,
    'savedShotsFromInsideTheBox' : x['statistics']['savedShotsFromInsideTheBox'] if 'statistics' in x.keys() and 'tosavedShotsFromInsideTheBoxalCross' in x['statistics'].keys() else np.NaN,
    'saves' : x['statistics']['saves'] if 'statistics' in x.keys() and 'saves' in x['statistics'].keys() else np.NaN,
    'totalKeeperSweeper' : x['statistics']['totalKeeperSweeper'] if 'statistics' in x.keys() and 'totalKeeperSweeper' in x['statistics'].keys() else np.NaN,
    'accurateKeeperSweeper' : x['statistics']['accurateKeeperSweeper'] if 'statistics' in x.keys() and 'accurateKeeperSweeper' in x['statistics'].keys() else np.NaN,
    'minutesPlayed' : x['statistics']['minutesPlayed'] if 'statistics' in x.keys() and 'minutesPlayed' in x['statistics'].keys() else np.NaN,
    'touches' : x['statistics']['touches'] if 'statistics' in x.keys() and 'touches' in x['statistics'].keys() else np.NaN,
    'rating' : x['statistics']['rating'] if 'statistics' in x.keys() and 'rating' in x['statistics'].keys() else np.NaN,
    'totalCross' : x['statistics']['totalCross'] if 'statistics' in x.keys() and 'totalCross' in x['statistics'].keys() else np.NaN,
    'aerialWon' : x['statistics']['aerialWon'] if 'statistics' in x.keys() and 'aerialWon' in x['statistics'].keys() else np.NaN,
    'duelWon' : x['statistics']['duelWon'] if 'statistics' in x.keys() and 'duelWon' in x['statistics'].keys() else np.NaN,
    'interceptionWon' : x['statistics']['interceptionWon'] if 'statistics' in x.keys() and 'interceptionWon' in x['statistics'].keys() else np.NaN,
    'totalTackle' : x['statistics']['totalTackle'] if 'statistics' in x.keys() and 'totalTackle' in x['statistics'].keys() else np.NaN,
    'fouls' : x['statistics']['fouls'] if 'statistics' in x.keys() and 'fouls' in x['statistics'].keys() else np.NaN,
    'shotOffTarget' : x['statistics']['shotOffTarget'] if 'statistics' in x.keys() and 'shotOffTarget' in x['statistics'].keys() else np.NaN,
    'wasFouled' : x['statistics']['wasFouled'] if 'statistics' in x.keys() and 'wasFouled' in x['statistics'].keys() else np.NaN,
    'onTargetScoringAttempt' : x['statistics']['onTargetScoringAttempt'] if 'statistics' in x.keys() and 'onTargetScoringAttempt' in x['statistics'].keys() else np.NaN,
    'blockedScoringAttempt' : x['statistics']['blockedScoringAttempt'] if 'statistics' in x.keys() and 'blockedScoringAttempt' in x['statistics'].keys() else np.NaN,
    'wonContest' : x['statistics']['wonContest'] if 'statistics' in x.keys() and 'wonContest' in x['statistics'].keys() else np.NaN,
    'totalContest' : x['statistics']['totalContest'] if 'statistics' in x.keys() and 'totalContest' in x['statistics'].keys() else np.NaN,
    'challengeLost' : x['statistics']['challengeLost'] if 'statistics' in x.keys() and 'challengeLost' in x['statistics'].keys() else np.NaN,
    'dispossessed' : x['statistics']['dispossessed'] if 'statistics' in x.keys() and 'dispossessed' in x['statistics'].keys() else np.NaN,
    'savedShotsFromInsideTheBox' : x['statistics']['savedShotsFromInsideTheBox'] if 'statistics' in x.keys() and 'savedShotsFromInsideTheBox' in x['statistics'].keys() else np.NaN,
    'totalOffside' : x['statistics']['totalOffside'] if 'statistics' in x.keys() and 'totalOffside' in x['statistics'].keys() else np.NaN,
    'bigChanceCreated' : x['statistics']['bigChanceCreated'] if 'statistics' in x.keys() and 'bigChanceCreated' in x['statistics'].keys() else np.NaN,
    'goals' : x['statistics']['goals'] if 'statistics' in x.keys() and 'goals' in x['statistics'].keys() else np.NaN }  
 for x in home['players']
]


In [ ]:
pd.DataFrame(data=rows)

In [ ]:
matches=[x for x in dp.data['events'] if 'coverage' in x.keys()]
matches=[x for x in matches if x['coverage']>-1]
matches=[x for x in matches if x['status']['code']==100 or x['status']['code']==110]
len(matches)

In [ ]:
a=[  {'tournament' : x['tournament']['slug'],
    'country' : x['tournament']['category']['slug'],
    'round' : x['roundInfo']['round'] if 'roundInfo' in x.keys() else np.NaN,
    'status' : x['status']['code'],
    'homeTeam' : x['homeTeam']['name'],
    'homeTeamShort' : x['homeTeam']['shortName'],
    'awayTeam' : x['awayTeam']['name'],
    'awayTeamShort' : x['awayTeam']['shortName'],
    'homeScoreFT' : x['homeScore']['normaltime'] if 'normaltime' in x.keys() else np.NaN,
    'homeScoreET' : x['homeScore']['current'] if 'current' in x.keys() else np.NaN,
    'homeScoreHT' : x['homeScore']['period1'] if 'period1' in x.keys() else np.NaN,
    'awayScoreFT' : x['awayScore']['normaltime'] if 'normaltime' in x.keys() else np.NaN,
    'awayScoreET' : x['awayScore']['current'] if 'current' in x.keys() else np.NaN,
    'awayScoreHT' : x['awayScore']['period1'] if 'period1' in x.keys() else np.NaN,
    'id' : x['id'],
    'startTimestamp' : x['startTimestamp'],
    'coverage' : x['coverage'],
    'winnerCode' : x['winnerCode']} for x in matches]

In [ ]:
pd.DataFrame(data=a)


In [ ]:
load_json('best-players',did, headers)

In [ ]:
did=9091942
load_match_info(did)

In [60]:
df_matches=pd.read_csv('data/matches_done.csv', index_col=None)
df_statistics=pd.read_csv('data/statistics.csv', index_col=None)
df_formations=pd.read_csv('data/formations.csv', index_col=None)
df_votes=pd.read_csv('data/votes.csv', index_col=None)

In [61]:
df=df_matches.merge(df_formations, on='mid', how='left')
df=df.merge(df_votes, on='mid', how='left')

df['v1_prob']=df['vote1']/(df['vote1']+df['vote2']+df['voteX'])
df['v2_prob']=df['voteX']/(df['vote1']+df['vote2']+df['voteX'])
df['v0_prob']=df['vote2']/(df['vote1']+df['vote2']+df['voteX'])

df['v1x_prob']=(df['vote1']+df['voteX'])/(df['vote1']+df['vote2']+df['voteX'])
df['v2x_prob']=(df['vote2']+df['voteX'])/(df['vote1']+df['vote2']+df['voteX'])
df['v12_prob']=(df['vote1']+df['vote2'])/(df['vote1']+df['vote2']+df['voteX'])

In [54]:
df.loc[df['homeScoreHT']>df['awayScoreHT'], 'res']=1
df.loc[df['homeScoreHT']<df['awayScoreHT'], 'res']=0
df.loc[df['homeScoreHT']==df['awayScoreHT'], 'res']=2
df['res1']=np.where(df['homeScoreHT']>df['awayScoreHT'],1,0)
df['res0']=np.where(df['homeScoreHT']<df['awayScoreHT'],1,0)
df['res2']=np.where(df['homeScoreHT']==df['awayScoreHT'],1,0)
df['res1x']=np.where(df['homeScoreHT']>=df['awayScoreHT'],1,0)
df['res2x']=np.where(df['homeScoreHT']<=df['awayScoreHT'],1,0)
df['res12']=np.where(~(df['homeScoreHT']==df['awayScoreHT']),1,0)

In [63]:
df.loc[df['homeScoreFT']>df['awayScoreFT'], 'res']=1
df.loc[df['homeScoreFT']<df['awayScoreFT'], 'res']=0
df.loc[df['homeScoreFT']==df['awayScoreFT'], 'res']=2
df['res1']=np.where(df['homeScoreFT']>df['awayScoreFT'],1,0)
df['res0']=np.where(df['homeScoreFT']<df['awayScoreFT'],1,0)
df['res2']=np.where(df['homeScoreFT']==df['awayScoreFT'],1,0)
df['res1x']=np.where(df['homeScoreFT']>=df['awayScoreFT'],1,0)
df['res2x']=np.where(df['homeScoreFT']<=df['awayScoreFT'],1,0)
df['res12']=np.where(~(df['homeScoreFT']==df['awayScoreFT']),1,0)

In [64]:
df[['res1','v1_prob']].corr()


,res1,v1_prob
res1,1.000000,0.296231
v1_prob,0.296231,1.000000


In [65]:
df[['res2','v2_prob']].corr()

,res2,v2_prob
res2,1.000000,0.063173
v2_prob,0.063173,1.000000


In [66]:
df[['res0','v0_prob']].corr()

,res0,v0_prob
res0,1.000000,0.295919
v0_prob,0.295919,1.000000


In [67]:
df[['res1x','v1x_prob']].corr()

,res1x,v1x_prob
res1x,1.000000,0.296427
v1x_prob,0.296427,1.000000


In [69]:
df[['res2x','v2x_prob']].corr()

,res2x,v2x_prob
res2x,1.000000,0.295779
v2x_prob,0.295779,1.000000


In [68]:
df[['res12','v12_prob']].corr()

,res12,v12_prob
res12,1.000000,0.063173
v12_prob,0.063173,1.000000


In [187]:
df

,homeTeamShort,awayTeam,tournament,awayTeamShort,homeTeam,startTimestamp,mid,awayScoreFT,awayScoreET,done,...,v1x_prob,v2x_prob,v12_prob,res,res1,res0,res2,res1x,res2x,res12
0,Western Sydney,Central Coast Mariners,a-league,Central Coast,Western Sydney Wanderers,1420101000,5578766,0.0,0,1,...,0.714829,0.520913,0.764259,2.0,0,0,1,1,1,0
1,Dundee Utd,Dundee FC,premiership,Dundee,Dundee United,1420114500,5587542,2.0,2,1,...,0.932605,0.209472,0.857923,1.0,1,0,0,1,0,1
2,Stoke,Manchester United,premier-league,Man Utd,Stoke City,1420116300,5582826,1.0,1,1,...,0.199892,0.901950,0.898158,2.0,0,0,1,1,1,0
3,İdman Yurdu,Fatih Karagümrük,turkiye-kupasi-group-c,Karagümrük,Mersin İdman Yurdu,1420119000,6378723,0.0,0,1,...,0.730769,0.461538,0.807692,2.0,0,0,1,1,1,0
4,Aston Villa,Crystal Palace,premier-league,Crystal Palace,Aston Villa,1420124400,5582812,0.0,0,1,...,0.872861,0.348166,0.778973,2.0,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7589,Tondela,CD Aves,primeira-liga,Aves,Tondela,1524409200,7473290,0.0,0,1,...,0.854422,0.471435,0.674143,2.0,0,0,1,1,1,0
7590,Balıkesirspor,Ümraniyespor,1-lig,Ümraniyespor,Balıkesirspor,1524409200,7507476,1.0,1,1,...,0.524116,0.752412,0.723473,0.0,0,1,0,0,1,1
7591,Rizespor,Giresunspor,1-lig,Giresunspor,Çaykur Rizespor,1524409200,7507466,0.0,0,1,...,0.928172,0.256185,0.815642,2.0,0,0,1,1,1,0
7592,Daco-Getica B.,Gaz Metan Mediaș,liga-1-relegation-round,Gaz Metan,Daco-Getica Bucureşti,1524409200,7736048,3.0,3,1,...,0.730769,0.647215,0.622016,0.0,0,1,0,0,1,1


In [2]:
df_incidents=pd.read_csv('data/incidents.csv', index_col=None)
df_incidents=df_incidents.loc[df_incidents['incidentType']!='varDecision']
df_incidents=df_incidents.loc[df_incidents['incidentType']!='injuryTime']
df_incidents=df_incidents.loc[df_incidents['time']>0]
df_incidents=df_incidents.loc[df_incidents['time']<91]

df_graph=pd.read_csv('data/graph.csv', index_col=None)
df_graph=df_graph.loc[df_graph['minute']>0]
df_graph=df_graph.loc[df_graph['minute']<91]

In [123]:
df_incidents['time'].unique()

array([20., 32., 45., 55., 56., 63., 67., 70., 72., 80., 83., 90., 29.,
       30., 46., 47., 57., 66., 71., 81., 82., 87., 37., 42., 61., 68.,
       69., 77., 79.,  9., 10., 31., 36., 44., 53., 60., 75., 89.,  6.,
       41., 58., 73., 88.,  2., 17., 52., 19., 35., 65., 74., 13., 25.,
       49., 54., 59., 62., 85., 43., 48., 50., 51.,  4., 39., 86., 18.,
       34., 21., 84., 38., 16., 24., 64., 23., 78.,  3., 22., 26., 76.,
       11., 33., 28.,  7., 40., 15., 12., 27.,  8.,  1., 14.,  5.])

In [117]:
df_incidents.loc[df_incidents['time']<-5]

,mid,player1,position1,player2,position2,incidentType,isHome,time,addedTime


In [121]:
df_incidents.loc[df_incidents['mid']==5555430]

,mid,player1,position1,player2,position2,incidentType,isHome,time,addedTime
3869,5555430,Steven Defour,M,NaN,NaN,yellow,False,32.0,NaN
3870,5555430,Jeff Louis,M,NaN,NaN,yellow,True,43.0,NaN
3871,5555430,NaN,NaN,NaN,NaN,period,NaN,45.0,999.0
3872,5555430,Steven Defour,M,NaN,NaN,yellowRed,False,53.0,NaN
3873,5555430,Sacha Kljestan,M,Oswal Alvarez,F,substitution,False,59.0,NaN
3874,5555430,Laurent Ciman,D,Adrien Trebel,M,goal,True,64.0,NaN
3875,5555430,Laurent Ciman,D,NaN,NaN,yellow,True,65.0,NaN
3876,5555430,Paul-José Mpoku,F,Geoffrey Bia,M,substitution,True,67.0,NaN
3877,5555430,Marko Marin,M,Maxime Colin,D,substitution,False,75.0,NaN
3878,5555430,Aaron Leya Iseka,F,Youri Tielemans,M,substitution,False,76.0,NaN


In [118]:
mids=df_incidents.loc[df_incidents['time']<0]['mid']

In [120]:
df_matches.loc[df_matches['mid'].isin(mids)]

,homeTeamShort,awayTeam,tournament,awayTeamShort,homeTeam,startTimestamp,mid,awayScoreFT,awayScoreET,done,...,homeScoreFT,winnerCode,homeScoreHT,round,country,status,ts,homeScoreET,awayScoreHT,id
67,Elazığspor,Adanaspor,1-lig,Adanaspor,Elazığspor,1420477200,5862734.0,0.0,0.0,1,...,0.0,3,0.0,16.0,turkey,100,2015-01-05 17:00:00+00:00,0.0,0.0,NaN
69,Balıkesirspor,Trabzonspor,super-lig,Trabzonspor,Balıkesirspor,1420480800,5722442.0,2.0,2.0,1,...,2.0,3,2.0,16.0,turkey,100,2015-01-05 18:00:00+00:00,2.0,0.0,NaN
119,Panthrakikos,Atromitos Athens,super-league,Atromitos,Panthrakikos,1420989300,6570346.0,1.0,1.0,1,...,1.0,3,1.0,11.0,greece,100,2015-01-11 15:15:00+00:00,1.0,0.0,NaN
150,Levadiakos,Asteras Tripolis,super-league,Asteras,Levadiakos,1421586000,5774462.0,1.0,1.0,1,...,3.0,1,1.0,19.0,greece,100,2015-01-18 13:00:00+00:00,3.0,1.0,NaN
171,Standard Liège,RSC Anderlecht,pro-league,Anderlecht,Standard Liège,1422192600,5555430.0,0.0,0.0,1,...,2.0,1,0.0,23.0,belgium,100,2015-01-25 13:30:00+00:00,2.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6988,Ümraniyespor,Eskişehirspor,1-lig,Eskişehirspor,Ümraniyespor,1518948000,7507373.0,1.0,1.0,1,...,0.0,2,0.0,22.0,turkey,100,2018-02-18 10:00:00+00:00,0.0,1.0,NaN
7183,Karpaty,Olimpik Donetsk,premier-league-relegation-round,Olimpik,Karpaty Lviv,1520769600,7743064.0,3.0,3.0,1,...,0.0,2,0.0,23.0,ukraine,100,2018-03-11 12:00:00+00:00,0.0,1.0,NaN
7203,Novara,Brescia,serie-b,Brescia,Novara,1520883000,7517123.0,1.0,1.0,1,...,2.0,1,0.0,30.0,italy,100,2018-03-12 19:30:00+00:00,2.0,1.0,NaN
7226,Rudar,NK Celje,prvaliga,Celje,Rudar Velenje,1521295200,7457734.0,1.0,1.0,1,...,2.0,1,1.0,24.0,slovenia,100,2018-03-17 14:00:00+00:00,2.0,1.0,NaN


In [17]:
df_matches

,homeTeamShort,awayTeam,tournament,awayTeamShort,homeTeam,startTimestamp,mid,awayScoreFT,awayScoreET,done,coverage,homeScoreFT,winnerCode,homeScoreHT,round,country,status,ts,homeScoreET,awayScoreHT
0,Western Sydney,Central Coast Mariners,a-league,Central Coast,Western Sydney Wanderers,1420101000,5578766,0.0,0,1,1,0.0,3,0,13.0,australia,100,2015-01-01 08:30:00+00:00,0,0
1,Dundee Utd,Dundee FC,premiership,Dundee,Dundee United,1420114500,5587542,2.0,2,1,1,6.0,1,4,19.0,scotland,100,2015-01-01 12:15:00+00:00,6,1
2,Stoke,Manchester United,premier-league,Man Utd,Stoke City,1420116300,5582826,1.0,1,1,1,1.0,3,1,20.0,england,100,2015-01-01 12:45:00+00:00,1,1
3,İdman Yurdu,Fatih Karagümrük,turkiye-kupasi-group-c,Karagümrük,Mersin İdman Yurdu,1420119000,6378723,0.0,0,1,1,3.0,1,0,4.0,turkey,100,2015-01-01 13:30:00+00:00,3,0
4,Aston Villa,Crystal Palace,premier-league,Crystal Palace,Aston Villa,1420124400,5582812,0.0,0,1,1,0.0,3,0,20.0,england,100,2015-01-01 15:00:00+00:00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7589,Tondela,CD Aves,primeira-liga,Aves,Tondela,1524409200,7473290,0.0,0,1,1,3.0,1,0,31.0,portugal,100,2018-04-22 15:00:00+00:00,3,0
7590,Balıkesirspor,Ümraniyespor,1-lig,Ümraniyespor,Balıkesirspor,1524409200,7507476,1.0,1,1,1,1.0,3,0,32.0,turkey,100,2018-04-22 15:00:00+00:00,1,1
7591,Rizespor,Giresunspor,1-lig,Giresunspor,Çaykur Rizespor,1524409200,7507466,0.0,0,1,1,1.0,1,0,32.0,turkey,100,2018-04-22 15:00:00+00:00,1,0
7592,Daco-Getica B.,Gaz Metan Mediaș,liga-1-relegation-round,Gaz Metan,Daco-Getica Bucureşti,1524409200,7736048,3.0,3,1,1,0.0,2,0,6.0,romania,100,2018-04-22 15:00:00+00:00,0,1
